# Agent_reader usage guide

### Basic usage

Let's import and initialize `SimpleReaderTool`.

In [1]:
%load_ext autoreload
%autoreload 2

from reader import SimpleReaderTool

simple_reader_tool = SimpleReaderTool()

To test the tool, let's create a minimal agent. To understand Langchain agents better, refer to Langchain documentation - [Getting Started with Agents](https://python.langchain.com/en/latest/modules/agents/getting_started.html).

In [3]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

# Make sure you've set the OPENAI_API_KEY environment variable
llm = ChatOpenAI(temperature=0)

tools = [simple_reader_tool]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

result = agent.run(
    "What does Langchain aim at," "based on https://blog.langchain.dev/about/ ?"
)
print(result)



> Entering new AgentExecutor chain...
I need to read the Langchain blog to find out what their aim is.
Action: read_page
Action Input: https://blog.langchain.dev/about/
Observation: 
TITLE: About LangChain
AUTHORS: []
PUBLISH DATE: 2023-01-16 06:31:39+00:00
TOP_IMAGE_URL: https://blog.langchain.dev/content/images/2023/01/wowow.png
TEXT:

Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. But using these LLMs in isolation is often not enough to create a truly powerful app - the real power comes when you are able to combine them with other sources of computation or knowledge.

LangChain is aimed at assisting in the development of those types of applications.

Thought:Based on the observation, Langchain aims at assisting in the development of applications that combine large language models with other sources of computation or knowledge.
Thought: I have found the answer to the question.
Final

# Advanced usage

`SimpleReaderTool` takes only a single input -- the page URL -- and so is perfect as a start. However, it also truncates any page contents above a limit, which makes it fail on longer pages.

`ReaderTool` can take multiple inputs. In addition to the URL, your agent can use the following arguments:
* `include_body` (defaults to `True`) - if set to False, only page metadata will be fetched.
* `cursor` - if a page is too long, the first call to `ReaderTool` will return a truncated page and a cursor to continue reading from. Using this mechanism, the agent can read the whole page in chunks by making successive calls to `ReaderTool` with increasing `cursor` values.

To use `ReaderTool`, your agent must support multi-input tools (also called [Structured tools](https://python.langchain.com/en/latest/modules/agents/agents/examples/structured_chat.html)).

In [7]:
from reader import ReaderTool

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

reader_tool = ReaderTool()

multi_tools = [reader_tool]

multi_agent = initialize_agent(
    multi_tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


result = multi_agent.run(
    "What quote does Paul Graham use from "
    "Zen and the Art of Motorcycle Maintenance, "
    "based on http://www.paulgraham.com/startupideas.html ?"
)
print(result)



> Entering new AgentExecutor chain...
I need to use the `read_page` tool to access the website http://www.paulgraham.com/startupideas.html and find the quote from Zen and the Art of Motorcycle Maintenance that Paul Graham uses. 

Action: 
```
{
  "action": "read_page",
  "action_input": {
    "url": "http://www.paulgraham.com/startupideas.html",
    "include_body": true
  }
}
```


Observation: 
TITLE: How to Get Startup Ideas
AUTHORS: []
PUBLISH DATE: None
TOP_IMAGE_URL: http://ycombinator.com/arc/arc.png
TEXT:









Want to start a startup? Get funded by Y Combinator.

November 2012



The way to get startup ideas is not to try to think of startup ideas. It's to look for problems, preferably problems you have yourself.



The very best startup ideas tend to have three things in common: they're something the founders themselves want, that they themselves can build, and that few others realize are worth doing. Microsoft, Apple, Yahoo, Google, and Facebook all began this way.



Pr

Thought:I need to use the `read_page` tool again to continue reading the website from where it was truncated. I will set the cursor to 8000.

Action: 
```
{
  "action": "read_page",
  "action_input": {
    "url": "http://www.paulgraham.com/startupideas.html",
    "include_body": true,
    "cursor": 8000
  }
}
```



Observation: y, the way to have good startup ideas is to become the sort of person who has them.



Being at the leading edge of a field doesn't mean you have to be one of the people pushing it forward. You can also be at the leading edge as a user. It was not so much because he was a programmer that Facebook seemed a good idea to Mark Zuckerberg as because he used computers so much. If you'd asked most 40 year olds in 2004 whether they'd like to publish their lives semi-publicly on the Internet, they'd have been horrified at the idea. But Mark already lived online; to him it seemed natural.



Paul Buchheit says that people at the leading edge of a rapidly changing field "

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4338 tokens. Please reduce the length of the messages.